In [145]:
import numpy as np
import cv2
import pandas as pd
import os
import pytesseract
from PIL import Image
import time
import concurrent.futures
pd.set_option('display.max_colwidth', None)
from collections import Counter
import json

In [146]:
def find_name(filepath):
    """This function reads the image, and returns the name of the boulder"""
    
    image = Image.open(filepath)
    bounding_box = (50,127,731,155) #name location
    cropped_image = image.crop(bounding_box)
    text = pytesseract.image_to_string(cropped_image)
    text = text[:-2]
    text = text.replace("@","") #Some Emojis come as @
    
    return text

In [147]:
def find_grade(filepath):
    """This function reads the image, and returns the grade of the boulder"""
    image = Image.open(filepath)
    bounding_box = (330,155,460,175) #grade location
    cropped_image = image.crop(bounding_box)
    text = pytesseract.image_to_string(cropped_image)


    text = text.partition("V") 
    text = text[2]
    text = text.strip()
    text = text.partition(" ") 
    text = text[0]
    text = text.replace("O","0") #V0's are read as VO
    
    return text

In [148]:
def make_hold_grid():
    hand_x = [54,114,173,233,293,352,412,471,530,590,650]
    hand_y = [176,235,295,354,414,473,533,592,652,711,771,831,890,950,1009,1069,1128,1188,1247]
    foot_left_x = [18,137,256,375,494,613]
    foot_left_y = [383,502,621,740,859,978,1097]
    foot_right_x = [77,196,315,434,554,672]
    foot_right_y = [323,442,561,680,799,918,1037,1156]
    foot_bottom_x = [17,77,137,196,256,315,375,434,494,553,613,672]
    foot_bottom_y = [1276]
    
    hands = make_hold_array(hand_x,hand_y)
    foot_left = make_hold_array(foot_left_x,foot_left_y)
    foot_right = make_hold_array(foot_right_x,foot_right_y)
    foot_bottom = make_hold_array(foot_bottom_x,foot_bottom_y)
    
    holds = np.append(hands,foot_left,axis=0)
    holds = np.append(holds,foot_right,axis=0)
    holds = np.append(holds,foot_bottom,axis=0)
    
    #sort holds, y value first then x, so we sort by row, then column
    holds = holds[np.lexsort((holds[:,0],holds[:,1]))]

    return holds

In [149]:
def make_hold_array(x_coordinates, y_coordinates):
    grid = np.meshgrid(x_coordinates,y_coordinates)
    grid = np.array(grid)
    holds = grid.reshape(2,-1).T
    
    return holds

In [150]:
def invert_element(key, value):
    """This function inverts data that looks like key:[value] to value[0]: key, value[1]: key...."""
    return {x: key for x in value}

In [151]:
def convert_dataframe(temp_data,name,grade,filepath):
    """This function creates a dataframe in the format of hold number as columns and hold type (start, foot, hand, finish)
    as the value, out of a dictionary and a grade. Dataframe is made with grade and name, which are given"""    
    inverted_data = {}
    
    data = {'name':name,
            'grade':grade,
            'filepath':filepath,
            }
    
    for key,value in temp_data.items():
        inverted_data.update(invert_element(key,value))
#         data.update(invert_element(key,value))
    data.update({f"Hold {key}":value for key,value in inverted_data.items()})
    
    return pd.DataFrame([data])

In [152]:
def processHoldType(img,hold_type,threshold):
    """This function does all the processing. Taking an image, hold type, and threshold and returning a list of(x,y) 
    coordinates for holds that match the hold type"""
    
    hold_length = 60  
    hold_height = 60  
    
    hold = cv2.imread(f"{hold_type}.png")
    hold_result = cv2.matchTemplate(img, hold, cv2.TM_CCOEFF_NORMED)
    hold_yloc, hold_xloc = np.where(hold_result >= threshold)
    hold_rectangles = [[x, y, hold_length, hold_height] for x,y in zip(hold_xloc, hold_yloc)]
    hold_rectangles, _ = cv2.groupRectangles(hold_rectangles, 1, 0.2)
    hold_rectangles = [[x, y] for x,y,_,_ in hold_rectangles]
    
    return np.array(hold_rectangles)

In [153]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

In [154]:
with open('hold_types.json', 'r') as f:
    hold_type_dict = json.load(f)
    
def count_hold_types(indexed_holds,hold_type_dict = hold_type_dict):
    nested_list = [hold_type_dict[str(x)] for x in indexed_holds]

    c = Counter()
    for x in nested_list:
        c.update(x)
        
    return dict(c)

In [155]:
count = 0#TEMP
def processImage(filepath):
    """This function takes a single image's file path, processes the image, and returns a dataframe, with the format hold number for columns
    and hold type as the value."""
    global count#TEMP
    print(count)#TEMP
    count +=1#TEMP
    
    img = cv2.imread(filepath)

    hand_result = processHoldType(img,'hand',.40)
    foot_result = processHoldType(img,'foot',.40)
    start_result = processHoldType(img,'start',.60)
    finish_result = processHoldType(img,'finish',.60)
    
    matches = {2: hand_result,
          3: foot_result,
          1: start_result,
          4: finish_result,
    }
    
    matches_indexes = ['None','Start','Hand','Foot','Finish']

    found_holds = {key: find_many_id(value) for key, value in matches.items()}
    
    name = find_name(filepath)
    grade = find_grade(filepath)
    
    found_dataframe = convert_dataframe(found_holds,name,grade,filepath) 
    
    return found_dataframe


In [156]:
holds = make_hold_grid()

def find_id(input_cord, holds = holds):
    """Given a coordinate (x,y) This function finds the closest hold out of a list of known holds, and returns the hold
    number"""
    
    x_dist = (input_cord[0] - holds[:, 0]) #All rows, 0th element in each row
    y_dist = (input_cord[1] - holds[:, 1]) #All rows, first element in each row
    distance = (x_dist**2) + (y_dist**2) #diagonal distance
    return np.argmin(distance)

def find_many_id(my_holds, holds = holds):
    """Given a list of hold coordinates, this function will return a list of hold numbers associated with the coordinates"""
    
    return [find_id(my_holds[i],holds) for i in range(len(my_holds))]

In [157]:
path = '../photos/'
images = [path+file for file in os.listdir(path)]
# images = images[:50]

start1 = time.time()

my_data = pd.DataFrame(columns=[f"Hold {item}" for item in range(0, 310)])


if __name__ == "__main__":
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(processImage,images)

for result in results:
    my_data = pd.concat([my_data,result])
    
# my_data.fillna('-',inplace = True) #Turn all blanks into '-' for better readability
my_data.fillna('0',inplace = True)
my_data.reset_index(inplace = True, drop=True)
# my_data.sort_index(axis=1) #Transpose?

end1 = time.time()


print ("Time elapsed:", end1 - start1)

00

2
3
3
5
6
7
8
8
10
11
12
12
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
3434

36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
78
80
81
82
83
84
85
86
87
8888

90
91
92
93
94
95
96
97
98
99
100
101101

103
104
105
105
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228228

230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
18731873

1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
19011901

1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
19231923

1925
1926
1927
1928
1929
19301930

1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
19441944

1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
19991999

2001
2002
2003
2004
2005
20062006

2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
20312031

2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
36073607

3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
36223622

3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704


In [158]:
# my_data.style.format({'filepath': make_clickable})


In [159]:
my_data

Hold 0 Hold 1 Hold 2 Hold 3 Hold 4 Hold 5 Hold 6 Hold 7 Hold 8 Hold 9  \
0         0      0      0      0      0      4      0      0      0      0   
1         0      0      0      0      0      4      0      0      0      0   
2         0      0      0      0      0      4      0      0      0      0   
3         0      0      0      0      0      0      0      4      0      0   
4         0      0      0      4      0      0      0      0      0      0   
...     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
5062      0      0      0      0      0      0      0      4      0      0   
5063      0      0      4      0      0      0      0      0      0      0   
5064      0      0      0      0      0      0      0      0      4      0   
5065      0      0      0      0      0      0      0      0      0      0   
5066      0      0      0      0      0      4      0      4      0      0   

      ... Hold 304 Hold 305 Hold 306 Hold 307 Hold 308 Hold 309  \
0     ...        0        0        0        0        0        0   
1     ...        3        0        0        0        3        0   
2     ...        0        3        3        0        3        0   
3     ...        0        0        0        0        0        3   
4     ...        0        0        0        0        0        0   
...   ...      ...      ...      ...      ...      ...      ...   
5062  ...        0        3        0        0        0        0   
5063  ...        0        3        3        0        0        0   
5064  ...        0        0        0        0        0        0   
5065  ...        0        0        0        0        0        0   
5066  ...        0        0        0        0        0        0   

                             name grade  \
0                      frog style     4   
1     FIRE-STARTER, FIRE-FINISHER     3   
2     INVEST SMALL FOR BIG GAINS.     3   
3                          Devin2     3   
4                          Devin3     5   
...                           ...   ...   
5062                      Pinball     4   
5063                  Tap Tap Tap     3   
5064                  kevin noooo     5   
5065            You Don’t Know Me     3   
5066             Floats Your Boat     3   

                                             filepath Hold 310  
0     ../photos/2022-01-25 22_11_58-Rock Climbing.png        0  
1     ../photos/2022-01-25 22_12_03-Rock Climbing.png        0  
2     ../photos/2022-01-25 22_12_08-Rock Climbing.png        0  
3     ../photos/2022-01-25 22_12_13-Rock Climbing.png        0  
4     ../photos/2022-01-25 22_12_18-Rock Climbing.png        0  
...                                               ...      ...  
5062  ../photos/2022-01-26 08_17_00-Rock Climbing.png        0  
5063  ../photos/2022-01-26 08_17_05-Rock Climbing.png        0  
5064  ../photos/2022-01-26 08_17_10-Rock Climbing.png        0  
5065  ../photos/2022-01-26 08_17_15-Rock Climbing.png        0  
5066  ../photos/2022-01-26 08_17_20-Rock Climbing.png        0  

[5067 rows x 314 columns]

In [160]:
my_data.shape

(5067, 314)

In [161]:
my_data.to_csv('kilter_lookup1.csv')